In [16]:
import os
import math

In [17]:
import cv2 as cv
import numpy as np
from sklearn import svm

from matplotlib import pyplot as plt

In [18]:
positivePath = os.path.join('.','cars','positive')
negativePath = os.path.join('.','cars','negative')

In [19]:
positiveFiles = os.listdir(positivePath)
negativeFiles = os.listdir(negativePath)

In [20]:
numPositive = len(positiveFiles)
numNegative = len(negativeFiles)

In [21]:
posLabel = np.ones(numPositive,dtype=np.uint8)
negLabel = np.zeros(numNegative,dtype=np.uint8)

In [22]:
labels = np.append(posLabel,negLabel)

In [23]:
features = []
j = 0
for file in positiveFiles :
    feature = []
    filePath = os.path.join(positivePath,file)
    image = cv.imread(filePath)
    #feature 1st - 2nd
    h,w,c = image.shape
    f1 = w/h
    f2 = math.log(w*h)
    #grayScale -> Binary Threshold -> Histogram
    imageGray = cv.cvtColor(image,cv.COLOR_RGB2GRAY)
    adaptive = cv.adaptiveThreshold(imageGray,255,
                                    cv.ADAPTIVE_THRESH_GAUSSIAN_C,
                                    cv.THRESH_BINARY,
                                    31,5)
    cv.imwrite('./cars/tmp2/{:03}.jpg'.format(j),adaptive)
    j+=1
    #histogram
    hist = np.histogram(adaptive)
    f3,f4 = hist[0][0],hist[0][-1]
    f3 = math.log(f3) if f3 != 0 else 0
    f4 = math.log(f4) if f4 != 0 else 0
    f5 = f3/f4 if f4 != 0 else 0
    features.append(np.asarray([f1,f2,f3,f4,f5]))

In [24]:
for file in negativeFiles:
    feature = []
    filePath = os.path.join(negativePath,file)
    image = cv.imread(filePath)

    #feature 1st - 2nd
    h,w,c = image.shape
    f1 = w/h
    f2 = math.log(w*h)
    #grayScale -> Binary Threshold -> Histogram
    imageGray = cv.cvtColor(image,cv.COLOR_RGB2GRAY)
    adaptive = cv.adaptiveThreshold(imageGray,255,
                                    cv.ADAPTIVE_THRESH_GAUSSIAN_C,
                                    cv.THRESH_BINARY,
                                    31,5)
    cv.imwrite('./cars/tmp2/{:03}.jpg'.format(j),adaptive)
    j+=1
    #histogram
    hist = np.histogram(adaptive)
    f3,f4 = hist[0][0],hist[0][-1]
    f3 = math.log(f3) if f3 != 0 else 0
    f4 = math.log(f4) if f4 != 0 else 0
    f5 = f3/f4 if f4 != 0 else 0
    features.append(np.asarray([f1,f2,f3,f4,f5]))

In [25]:
clf = svm.SVC(kernel='rbf')

In [26]:
clf.fit(features,labels)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [27]:
clf.score(features,labels)

0.9436619718309859

In [28]:
from sklearn.externals import joblib
joblib.dump(clf, 'plate2.pkl') 

['plate2.pkl']

In [30]:
clf.predict(features[40:60])

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=uint8)